In [52]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import librosa
import pickle

In [53]:
#Ścieżka do pliku tekstowego, w którym znajdują się informacje o mówcach
file_path = 'C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/SPEAKERS.TXT'

#Tworzę ramkę danych, w której znajdują się informacje o płci danego mówcy
df = pd.read_csv(file_path, sep='|', comment=';', usecols=[0, 1], 
                 names=["ID", "SEX"])



#Ścieżka do folderu, w którym znajdują się katalogi z nagraniami osób
file_path = 'C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100/'

#Wyciągam wszystkie nazwy podfolderów z powyższej ścieżki (są to ID nagranych osób)
subfolders = [f.name for f in os.scandir(file_path) if f.is_dir()]

#Sortuję ID nagranych osób (najpierw muszę zamienić ID na liczbę)
subfolders = sorted([int(item) for item in subfolders])
subfolders = np.array(subfolders)

df = df.loc[np.isin(np.array(df['ID']), subfolders)]

#Tworzę oddzielne ramki dla kobiet i mężczyzn
df_woman = df[df['SEX'] == ' F '].reset_index(drop=True)
df_man = df[df['SEX'] == ' M '].reset_index(drop=True)

In [54]:
#wyciągam foldery spekerów 
folders_path = "C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100"
subfolders = [f.name for f in os.scandir(folders_path) if f.is_dir()]
subfolders = sorted([int(item) for item in subfolders])
subfolders = [str(item) for item in subfolders]
#np. C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100/19
#gdzie 19 to ID

#tworzę łączę C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100 z ID aby uzyskać efekt jak pokazałem wyżej
#np. C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100/19
paths_with_ID = [folders_path + '/'+ subfolder for subfolder in subfolders]

#tworzę puste ramki aby przechowywać ID osoby i łączny czas nagrania
data_frame_for_duration_woman = pd.DataFrame(columns=['ID', 'duration'])
data_frame_for_duration_man = pd.DataFrame(columns=['ID', 'duration'])

#w pętli liczę łączny czas nagrań wszystkich osób 
for path_with_ID in paths_with_ID:

    paths_inside_ID = [f.name for f in os.scandir(path_with_ID) if f.is_dir()]

    full_paths_to_files = [path_with_ID + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    all_files_for_ID = []
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files


    duration_in_seconds = 0
    for file_for_ID in all_files_for_ID:
        with sf.SoundFile(file_for_ID) as f:
            frames = len(f)  
            sample_rate = f.samplerate  
        duration = frames / sample_rate
        duration_in_seconds = duration_in_seconds + duration
    

    ID = path_with_ID.split('/')[-1]
    new_record = [ID, duration_in_seconds]
    
    if np.isin(ID, df_woman['ID']):
        data_frame_for_duration_woman.loc[len(data_frame_for_duration_woman)] = new_record
    else:
        data_frame_for_duration_man.loc[len(data_frame_for_duration_man)] = new_record

    print(ID)
    clear_output(wait=True)

#sortuję dane tak aby łączny czas nagrania był malejący
data_frame_for_duration_woman = data_frame_for_duration_woman.sort_values(by='duration', ascending=False)
data_frame_for_duration_man = data_frame_for_duration_man.sort_values(by='duration', ascending=False)

#wybieram 50 mężczyzn i 50 kobiet których łączny czas nagrań był najdłuższy
top_50_man = data_frame_for_duration_man.head(50)
top_50_woman = data_frame_for_duration_woman.head(50)

8975


In [55]:
folders_path = "C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100"

#tworzę ścieżki do osób których łączny czas nagrań jest najdłuższy
top_50_man_paths = [folders_path + '/' + ID for ID in top_50_man['ID']]
top_50_woman_paths = [folders_path + '/' + ID for ID in top_50_woman['ID']]

top_50_man_and_woman = top_50_man_paths + top_50_woman_paths

In [56]:
all_files = []
audio_owner = []

for i in range(0,100):

    ID = top_50_man_and_woman[i].split('/')[-1]
    
    #teraz jestem w folderze z ID np. C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100/19
    paths_inside_ID = [f.name for f in os.scandir(top_50_man_and_woman[i]) if f.is_dir()]

    #teraz jestem w folderze z danej osoby np. C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100/19/198
    full_paths_to_files = [top_50_man_and_woman[i] + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]
        
    

    #w pętli wyciągam wszystkie ścieżki do nagrań danej osoby czyli np.
    #C:/Users/zbugo/Desktop/praktyki_zadania/8/train-clean-100/LibriSpeech/train-clean-100/19/198/19-198-0000.flac
    #w all_files_for_ID przechowuje ścieżki do wszysktich plików danej osoby
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        audio_owner.extend([ID] * len(files))
        all_files = all_files + files

In [57]:
with open('C:/Users/zbugo/Desktop/praktyki_zadania/12/gmm.pkl', 'rb') as file:
    UBM = pickle.load(file)

c:\Users\zbugo\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GaussianMixture from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [58]:
#Liczba współczynników, które otrzymamy po przeprowadzeniu MFCC — ta ilość współczynników powstaje w wyniku DCT (dyskretnej transformacji kosinusowej)
quantity_of_mel_coef = 13
#Liczba filtrów melowych — ilość 'czapek', które zostaną nałożone na sygnał. Dla każdego z nich będzie sumowana energia
quantity_of_mel_filters = 26
#próbkowanie
sr = 16000 

mean_predict_proba_df = pd.DataFrame()

iteration = 0

#w pętli liczę embeddingi nagrań trenujących model UBM
for file in all_files:
    signal, sr = librosa.load(file, sr = sr)
    mfcc_signal = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=quantity_of_mel_coef, n_mels = quantity_of_mel_filters).T
    mfcc_signal = pd.DataFrame(mfcc_signal)
    embedding = pd.DataFrame(pd.DataFrame(UBM.predict_proba(mfcc_signal)).mean()).T
    mean_predict_proba_df = pd.concat([mean_predict_proba_df, embedding], ignore_index=True)

    iteration = iteration + 1

    print(iteration/len(all_files))
    clear_output(wait=True)
  

1.0


In [62]:
#nadaję indeksu, w taki sposób aby było wiadamo kto jest właścicielem nagrania, a właścieiwe embeddingu
mean_predict_proba_df.index = audio_owner

In [68]:
mean_predict_proba_df.to_pickle('mean_predict_proba_df.pkl')